# DESeq2 teoria

Parametrit
* $i \in \{1, \dots, n\}$, geenin indeksi (gene index)
* $j \in \{1, \dots, m\}$, näytteen indeksi (sample index)
* $r \in \{0, \dots, p-1\}$, kovarianssiindeksi (covariate index, with intercept r = 0)
* $K_{ij}$ geenin $i$ sekvenssilukumäärä näytteesä $j$ (counts of reads for gene i, sample j)
* $\mu_{ij}$ sovitettu keskiarvo (fitted mean)
* $\alpha_i$ geenikohtainen hajonta (gene-specific dispersion)
* $s_j$ näytekohtainen suuruustekijä (sample-specific size factor)
* $s_{ij}$ geeni- ja näytekohtainen normalisaatiovakio (gene- and sample-specific normalization factor)
* $q_{ij}$ proportional to true concentration of fragments
* $x_{jr}$ elements of the design matrix $X$
*

$$
NB(y | \mu, \alpha) = {y + \alpha -1 \choose y}(\frac{\mu}{\mu + \alpha})^n (\frac{\alpha}{\mu + \alpha})^\alpha
$$

### Tilastollinen malli ja normalisaatio

Sekvenssilukumäärä $K_{ij}$ geenille $i$ näytteessä $j$ mallinnetaan negatiivisen binomijakauman yleistetyllä lineaarisella mallilla:

$$
K_{ij} \sim NB(\mu_{ij}, \alpha_i)
$$

$$
\mu_{ij} = s_{ij}q_{ij}
$$

$$
\log q_{ij} = \sum_r x_{jr} \beta_{ir}
$$

Oletusarvoisesti normalisaatiovakiot $s_{ij}$ ovat vakioita yhdessä näytteessä, eli $s_{ij} = s_j$. Normalisaatiovakiot estimoidaan osamäärien mediaanimenetelmällä:

$$
s_j = \textrm{median}\frac{K_{ij}}{K_i^R}, K_i^R \neq 0 
$$
jossa
$$
K_i^R = (\prod_{j=1}^m K_{ij})^{1/m}
$$

### Expanded design matrices

Lineaarisessa mallinnuksessa luokkamuuttuja tai tekijä voi saada kaksi tai useampaa arvoa tai tasoa. Yleensä sunnittelumatriisissa yksi luokka on viitearvo (base level) ja se liitetään interceptiin. Standardi GLM:ssä viitetason valinta ei vaikuta kontrasteihin (LFCs). Tämä ei kuitenkaan päde tässä, kun käytetään ridge-regression-like shrinkage vakioolle, kun luokkamuuttjalla on enemmän kuin kaksi luokkaa. Tämä johtuu siitä, että viitearvoon ei kohdistu shrinkage, mutta muihin tasoihin kyllä.  

### Contrasts

Kontrastit tasojen välillä ja kontrastien keskivirheet voidaan laskea tavanomaisesti

$$
\beta_i^c = \overrightarrow{c} ^t \overrightarrow{\beta}_i
$$

$$
\textrm{SE}(\beta_i^c) = \sqrt{\overrightarrow{c}^t \sum_i \overrightarrow{c}}
$$

jossa $\overrightarrow{c}$ on numeerinen kontrasti, esim. 1 tai -1, ja määrittävät nimittäjän ja osittajan yksinkertaisessa kahden tason kontrasistssa. $\sum_i = \Cov(\overrightarrow{\beta}_i)$ määritetään alempana.

### Dispersion estimointi

Dispersioparamteri $\alpha_i$ noudattaa log-normaali priorijakuamaa, joka on keskitetty trendin suhteen, joka riippuu geenin keskimääräisestä, normalisoidusta lukumäärästä. 

$$
\log \alpha_i \sim N (\log \alpha_{tr}(\bar{\mu}_i), \sigma_d^2)
$$

jossa $\alpha_tr$ on funktio geenin kesimääräisestä, normalisoidusta lukumäärästä:

$$
\bar{\mu}_i = \frac{1}{m}\sum_j \frac{K_{ij}}{s_{ij}}
$$

Parametri kertoo priorin keskiarvoriippuvaisen odotusarvon. $\sigma_d$ on priorin leveys, eli tämä hyperparametri kertii kuinka paljon yksittäisen geenin dispersio eroaa trendistä. Trendifunktiolle käytetään paramtrisointia

$$
\alpha_{tr} = \frac{\alpha_1}{\bar{\mu}} + \alpha_0 \ (6)
$$

Dispersion esitmaatit saadaan tästä mallista kolmessa vaihessa. Menetelmä on nopea approksimaatio bayesiläiseen päättelyyn. Aluksi käytetään jokaisen geenin lukumäärää erikseen alustavan geenikohtaisen dispersioestimaatin $\alpha_i^{gw}$ korkeimman uskottavuuden menetelmällä. Tämän jälkeen sovitetaan dispersiotrendi $\alpha_{tr}$. Lopulta uskottuavuus yhdistetään trendin prioriin ja saadaan suurin a posterior arvo (MAP) lopullisena disperion estimaattina. Kolmen vaiheen yksityiskohdat alla.

### Geenikohtaisen hajonnan estimointi

Geenikohtainen dispersioestimaatti geenille $i$ estimoidaan sovittamalla negatiivisen binomijakauman yleistetty lineaarinen malli matriisiin X, joka kuvaa geenin lukumäärävastetta. Tämä malli käyttää karkeaa momenttimenetelmäestimointia dispersion estimoimiseksi. Estimointi perustuu ryhmän sisäiseen variansiin ja keskiarvoihin. Tämä alkuperäinen malli on välttämätön ensimmäisten sovitteiden aikaansaamiseksi, $\hat{\mu}_{ij}^0$. Tämän jälkeen maksimoidaan Cox-Reid adjusted likelihood of the dispersion, conditioned on the fitted values $\hat{\mu}_{ij}^0$ alkuperäisestä sovituksesta, ja saadaan geenikohtainen estimaatti $\alpha_i^{gw}$ eli

$$
\alpha_{i}^{gw} = \textrm{arg} \textrm{max}_{\alpha} l_{CR} (\alpha; \overrightarrow{\mu}_i^0, \overrightarrow{K}_i)
$$

jossa

$$
l_{CR}(\alpha; \overrightarrow{\mu}, \overrightarrow{K}) = l(\alpha) - \frac{1}{2}\log (\det(X^tWX))
$$
$$
l(\alpha) = \sum_j \log f_{NB}(K_j; \mu_j, \alpha)
$$

jossa $f_{NB}(k; \mu, \alpha)$ on negatiivisen binomijakauman pistetodennäköisyysfunktio keskiarvolla $\mu$ ja hajonnalla $\alpha$. Toinen termi on Cox-Reid bias adjustment. Tämä korjaus korjaa hajontaestimaattien negatiivisen biaksen, kun käytetään suurimman uskottavuuden estimaatteja sovitteille $\hat{\mu}_{ij}^0$. Se muodostetaan sovitteiden Fisherin informaatiosta, ja se lasketaan $\det(X^t W X)$, jossa $W$ on diagonal weight matrix from the standard iteratively reweighted least-squares algorithm. Yleistetyn lineaarisen mallin linkkifunktio on $g(\mu) = \log(\mu)$ ja sen varianssifunktio on $V(\mu; \alpha) = \mu + \alpha \mu^2$, ja matriisin alkiot $W_i$ ovat:

$$
w_{jj} = \frac{1}{g'(\mu_j)^2 V(\mu_j)} = \frac{1}{1/\mu_j + \alpha}
$$

Optimointi suoritetaan asteikolla $\log \alpha$ käyttämällä backtracking search with accepted proposals that satisfy Armijo conditions.

### Dispersion trendi